## 1.Introduction to PyTorch, a Deep Learning Library

### 1-1. Tensors
- Similar to array or matrix
- Building block of neural networks 
##
- Can be added or substracted, provided that their shapes are compatible
- a * b : element-wise multiplication vs a @ b = Matrix multiplication <br>

        → Perform addition and multiplication to process data and learn patterns 

In [ ]:
# Import torch
import torch

temperatures = [[72, 75, 78], [70, 73, 76]] # can be created from Python lists or Numpy arrays

# Create a tensor from temperatures
temperatures= torch.tensor(temperatures)

print(temperatures)


tensor([[72, 75, 78],
        [70, 73, 76]])


### 1-2. Cheking and adding tensors

In [9]:
adjustment = torch.tensor([[2, 2, 2], [2, 2, 2]])

# Display the shape of the adjustment tensor
print("Adjustment shape:", adjustment.shape)

# Display the type of the adjustment tensor

print("Adjustment type:", adjustment.dtype)

print("Temperatures shape:", temperatures.shape)
print("Temperatures type:", temperatures.dtype)


Adjustment shape: torch.Size([2, 3])
Adjustment type: torch.int64
Temperatures shape: torch.Size([2, 3])
Temperatures type: torch.int64


In [10]:
adjustment = torch.tensor([[2, 2, 2], [2, 2, 2]])

# Add the temperatures and adjustment tensors
corrected_temperatures = adjustment + temperatures
print("Corrected temperatures:", corrected_temperatures)


Corrected temperatures: tensor([[74, 77, 80],
        [72, 75, 78]])


### 1-3. Neural networks and layers
- A neural network consists of input(dataset features), hidden, and output layers(predictions).
- ''Fully connected network''(= linear model) 
    - Every input neuron connects to every output neuron 
    - Network with no hidden layers where the output layer is a linear layer
    - Linear layer : input @ wieghts + bias = output


- weight : Reflects the importance of different features
- bias : Provides the neuron with a baseline output 

ex) 
- input (temperature, humidity, wind) → output (rain, cloudy) <br>
humidity feature will have a more significant weight<br>
bias is to account for baseline information


In [12]:
# Linear layer network

import torch
import torch.nn as nn

input_tensor = torch.tensor([[0.3471, 0.4547, -0.2356]])

# Create a Linear layer 
linear_layer = nn.Linear (
    in_features= 3,
    out_features= 2
)

# Pass input_tensor through the linear layer 
output = linear_layer(input_tensor)

print(output)

tensor([[-0.3970,  0.1681]], grad_fn=<AddmmBackward0>)


### 1-4. Hidden layer and parameters
- Stacking layes with nn.Seqential()

Layer are made of neurons 
- Fully connected : each neuron links to all neurons in the previous layer 
- A neuron in a linear layer :
    - perfroms a linear operation using all neurons from the previous layer 
    - Has N+1 parameters : N from inputs and 1 for the bias

In [14]:
import torch 
import torch.nn as nn

input_tensor = torch.Tensor([[2, 3, 6, 7, 9, 3, 2, 1]])

# Create a container for stacking linear layers
model = nn.Sequential(
    nn.Linear(8,4),
    nn.Linear(4,1)
) # cf) nn.Linear(input features, output features)

output = model(input_tensor)
print(output)


tensor([[-2.0686]], grad_fn=<AddmmBackward0>)


In [19]:
# Counting the number of parameters

model = nn.Sequential(nn.Linear(9, 4),
                      nn.Linear(4, 2),
                      nn.Linear(2, 1))

total = 0
for parameter in model.parameters():
    print(parameter)  # Weight & bias
    total += parameter.numel()
    print(total)
print(total)


Parameter containing:
tensor([[-0.2275,  0.2435, -0.0524,  0.3298,  0.3296,  0.2640, -0.2381, -0.2140,
          0.0264],
        [-0.3031,  0.0515, -0.0308, -0.0951,  0.1180,  0.1785, -0.1220, -0.0733,
          0.0759],
        [-0.1972,  0.2063, -0.0784, -0.3155,  0.0543, -0.0286,  0.2107, -0.0309,
         -0.1947],
        [-0.2835,  0.2321,  0.0301, -0.2333,  0.2362,  0.2412,  0.0559, -0.1665,
         -0.1575]], requires_grad=True)
36
Parameter containing:
tensor([-0.0622,  0.3263, -0.0052, -0.0351], requires_grad=True)
40
Parameter containing:
tensor([[-0.0515, -0.4406,  0.4247, -0.3446],
        [-0.2672, -0.3528, -0.1264, -0.1666]], requires_grad=True)
48
Parameter containing:
tensor([-0.2156, -0.3809], requires_grad=True)
50
Parameter containing:
tensor([[-0.0717,  0.0155]], requires_grad=True)
52
Parameter containing:
tensor([0.4634], requires_grad=True)
53
53


----

## Week 1 review

###  Section 1: Implementing a simple linear NN

In [ ]:
class Net(nn.Module):
    def __init__(self) : 
        super(Net, self).__init__()  # Initalise parent class
        self.fc1 = nn.Linear(2,1)

    def forward(self, x):  # The function we specify which function to apply on the input 
        x = self.fc1(x)   # 입력값에 가중치 곱하고 편향 더해서 계산하는 것 
        return x

In [24]:
net1 = Net()

list(net1.parameters())

# requires_grad=True : 자동 미분 속성 
# → Backpropagation 알고리즘을 실행할 경우, Pytorch가 이 매개변수들을 최적화(학습)할 것임을 의미한다
# → 매개변수를 최적화 하려면 손실 값 (loss value)에 따른 기울기를 저장해야하는데 이 때 이 키워드가 필요한다

[Parameter containing:
 tensor([[-0.6601, -0.0942]], requires_grad=True),
 Parameter containing:
 tensor([-0.2797], requires_grad=True)]

In [28]:
# testing whether nerons works
x_input = torch.tensor([[0.1, 0.5]])
net1(x_input)

tensor([[-0.3928]], grad_fn=<AddmmBackward0>)

In [ ]:
# (0.1* - 0.6601) + (0.5 * -0.0942) + -0.2797

-0.39281

### Section 2: Implementing a perceptron in PyTorch


In [ ]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc1 = 
    
    def forward(self, x):
        x = self.fc1(x)
        x = 
        return x 

In [29]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        
        # Gap 1: single linear unit
        self.fc1 = nn.Linear(2, 1)
        
        # Hardcoded weights and bias
        with torch.no_grad():
            self.fc1.weight = nn.Parameter(torch.tensor([[1.0, 1.0]]))
            self.fc1.bias   = nn.Parameter(torch.tensor([-1.5]))
        
    def forward(self, x):
        x = self.fc1(x)
        
        # Gap 2: Heaviside step function
        x = (x > 0).float()
        return x

In [30]:
my_perceptron = Perceptron()
my_perceptron.fc1.weight.data = torch.tensor([[ 0.4, 0.2]])
my_perceptron.fc1.bias.data = torch.tensor([-0.3])

In [31]:
test_inputs = torch.tensor([
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0]
])

outputs = my_perceptron(test_inputs)

for x, y in zip(test_inputs, outputs):
    print(f"Input {x.tolist()} -> Output {int(y.item())}")

Input [0.0, 0.0] -> Output 0
Input [0.0, 1.0] -> Output 0
Input [1.0, 0.0] -> Output 1
Input [1.0, 1.0] -> Output 1


## Section 3: Solving a classification problem using a Perceptron

In [ ]:
class ExNet(nn.Module):
    def __init__(self):
        super(ExNet, self).__init__()
        self.fc1 = nn.Linear(2,1)
        
        self.fc1.weights = torch.nn.Parameter(torch.tensor([[0.66],[1]])) # 둘다 그림에서 계산한거 
        self.fc1.bias = torch.nn.Parameter(torch.tensor([[-0.5]]))  
        
        self.heaviside = torch.heaviside
        
        
    def forward(self, x):
        x = self.fc1(x)
        output = self.heavside(x, 0.5)
        return output

## Section 4: Training a perceptron
- implement a method that trains the modelsuing some training data + test

In [32]:
data = [(0.7,0.3,1),(0.4,0.5,1), (0.6,0.9, 1), (0.2,0.2, 0), (0.1, 0.1, 0)]

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2,1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.heaviside(x, torch.Tensor(1))
        return x


In [33]:
training_data = torch.Tensor(
    [(0.7,0.3, 1),
     (0.4,0.5, 1),
     (0.6,0.9, 1),
     (0.2,0.2, 0)]
)
learning_rate = 0.1

# Initialise bias, weight1, weight2 
initial_weights = torch.Tensor( (-0.5, 0.3, -0.2) )

In [34]:
# True : if the network fails to classify every sample in the data correctly.
# Whenever it returns True, it will imply that our network requires more training.
def keep_training(nn, data):
    for sample in data:
        if not torch.eq(nn(sample[0:2]), sample[-1]):
            return True
    return False

In [ ]:
# write a function to implement the learing algorithm of page 24 
def train_perceptron(learning_rate, initial_weights, data):
    perceptron = Net()# Gap 1
    perceptron.fc1.bias.data = # Gap 2
    perceptron.fc1.weight.data = # Gap 3
    
    
    while keep_training(perceptron, data):
        for sample in data:
            temp_output = # Gap 4
            label = sample[-1]
            delta_w = # Gap 5
            print(delta_w)
            perceptron.fc1.bias.data = # Gap 6
            perceptron.fc1.weight.data = # Gap 7
            
    return perceptron     

In [35]:
def train_perceptron(learning_rate, initial_weights, data):
    perceptron = Net()                     # Gap 1
    
    # initial_weights = (bias, w1, w2)
    perceptron.fc1.bias.data = initial_weights[0:1]           # Gap 2
    perceptron.fc1.weight.data = initial_weights[1:].view(1,2)  # Gap 3
    
    while keep_training(perceptron, data):
        for sample in data:
            x = sample[0:2]
            label = sample[-1]
            
            temp_output = perceptron(x)                         # Gap 4
            
            error = label - temp_output
            delta_w = learning_rate * error * x                 # Gap 5
            print(delta_w)
            
            perceptron.fc1.bias.data += learning_rate * error   # Gap 6
            perceptron.fc1.weight.data += delta_w.view(1,2)     # Gap 7
            
    return perceptron

In [36]:
# Let us now run our algorithm, and see how far it goes in solving or classification problem!

train_perceptron(0.1, initial_weights, training_data)

tensor([0.0700, 0.0300], grad_fn=<MulBackward0>)
tensor([0.0400, 0.0500], grad_fn=<MulBackward0>)
tensor([0.0600, 0.0900], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([0.0400, 0.0500], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([-0.0200, -0.0200], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([0.0400, 0.0500], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([-0.0200, -0.0200], grad_fn=<MulBackward0>)


Net(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
)

**Question:** *Compute the generalisation error of the perceptron trained above w.r.t. the ideal classification given in the above figure, i.e.  draw the classification boundary for the trained perceptron and compare it with that of the figure.*